<a href="https://colab.research.google.com/github/abhinav-bagwari/Malaria-Disease-Detection-using-Vgg19/blob/main/Malaria_Disease_Detection_using_Vgg19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Sequential, Model
from keras.applications.vgg19 import VGG19
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [ ]:
IMAGE_SIZE = [224, 224]
train_path = '/content/drive/MyDrive/Colab Notebooks/Dataset/Train'
test_path = '/content/drive/MyDrive/Colab Notebooks/Dataset/Test'

In [ ]:
# add preprocessing layer to the front of VGG
# imagenet weight already able to categorize thousands of categories
# input_shape = 224,224,3
# include_top = FALSE -> dropping the first and last layer coz last layer will have only 2 categories i.e parasitized and Uninfected
vgg = VGG19(input_shape = IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [ ]:
# don't train existing weights since it is already trained to classify multiples of 
#categories we need not to further updates the weights of vgg19 pre-trained model
for layer in vgg.layers:
  layer.trainable = False

In [ ]:
# to check how many classes is present inside the Train i.e Parasitized and Uninfected
folders = glob('/content/drive/MyDrive/Colab Notebooks/Dataset/Train/*')

In [ ]:
# Addding layers
x = Flatten()(vgg.output)
prediction  = Dense(len(folders),activation='softmax')(x)

In [ ]:
#Creating model object
model = Model(inputs = vgg.input, outputs = prediction)

In [ ]:
# the structure of the model
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:
model.compile(optimizer = "adam", loss ="categorical_crossentropy", metrics=['accuracy'])

In [ ]:
#Using the ImageDataGenerator to load images for Image Agumentation
train_datagen = ImageDataGenerator(rescale = 1/255,
                                   shear_range=0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1/255)

In [ ]:
training_set = train_datagen.flow_from_directory(directory = train_path,
                                                 target_size = (224,224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

testing_set = test_datagen.flow_from_directory(directory = test_path,
                                                 target_size = (224,224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 416 images belonging to 2 classes.
Found 134 images belonging to 2 classes.


In [ ]:
#fitting model
model.fit_generator(training_set,
                    validation_data = testing_set,
                    epochs = 5,
                    steps_per_epoch = len(training_set),
                    validation_steps = len(testing_set))

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
13/13 [==============================] - 286s 21s/step - loss: 1.1160 - accuracy: 0.5769 - val_loss: 0.7350 - val_accuracy: 0.6791
Epoch 2/5
13/13 [==============================] - 219s 17s/step - loss: 0.6611 - accuracy: 0.6827 - val_loss: 0.7107 - val_accuracy: 0.6119
Epoch 3/5
13/13 [==============================] - 221s 17s/step - loss: 0.4255 - accuracy: 0.7885 - val_loss: 0.4506 - val_accuracy: 0.7164
Epoch 4/5
13/13 [==============================] - 220s 17s/step - loss: 0.3567 - accuracy: 0.8389 - val_loss: 0.5849 - val_accuracy: 0.6642
Epoch 5/5
13/13 [==============================] - 219s 17s/step - loss: 0.3026 - accuracy: 0.8726 - val_loss: 0.4572 - val_accuracy: 0.7612


* **epochs**: an integer and number of epochs we want to train our model for

* **steps_per_epoch**: it specifies the 
total number of steps taken before
one epoch has finished and started the next epoch. By default it values is set to NULL.

* **validation_steps** :only if the validation_data is a generator then only this argument
can be used. It specifies the total number of steps taken from the generator before it is 
stopped at every epoch and its value is calculated as the total number of validation data points
in your dataset divided by the validation batch size


In [ ]:
model.save("malaria_model.h5")

In [ ]:
path = "/content/drive/MyDrive/Colab Notebooks/Dataset/Test/Uninfected/C3thin_original_IMG_20150608_163047_cell_26.png"
img = image.load_img(path, target_size=(224,224))

img = image.img_to_array(img)/255
img = np.array([img])
img.shape
# model.predict_classes(img)
pred = (model.predict(img) > 0.5).astype("int32")

In [ ]:
y = pred[0]

In [ ]:
y[0]

0

In [ ]:
if (y[0] == 0):
  result = "Uninfected"
else:
  result = "Infected"

print(result)

Uninfected
